# Sustinable Azure - Multi hosting with frondoor

This interactive notebook will guide you how to use Azure Front Door to switch hosting.

## How-To start

* Configure a Front Door (Classic) with a Back-End-Pool.
* The Back-End-Pool contains two aquivalent services with different hosting types
  * Azure App Service
  * Azure Function with consumption plan

# Install modules

In [ ]:
# Install-Module -Name az -Verbose
# Install-Module Az.FrontDoor
# Install-Module Az.Websites
# Install-Module Az.Functions
# Install-Module Az.Cdn
# Import-Module Az.Cdn

# Load account settings from file & initialize

In [ ]:
$CurrentDir = Get-Location
$SettingsFile = "AccountSettings.json"

$AccountSettingsFile = Join-Path -Path $CurrentDir -ChildPath $SettingsFile
if (-not (Test-Path $AccountSettingsFile))
{
    Copy-Item "$AccountSettingsFile.example" $AccountSettingsFile -ErrorAction 'Stop'
}

$Settings = Get-Content $AccountSettingsFile | ConvertFrom-Json
$TenantId = $Settings.TenantId
$AccountId = $Settings.AccountId
$SubscriptionId = $Settings.SubscriptionId
$ResourceGroupName = $Settings.ResourceGroupName
$FrontDoorName = $Settings.FrontDoorName
$WebAppAddress = $Settings.WebAppAddress
$FunctionAddress = $Settings.FunctionAddress
$WebAppName = $Settings.WebAppName
$FunctionName = $Settings.FunctionName


## Connect Azure

In [ ]:
az account set --subscription $SubscriptionId

## Scale Down

We will deactivate the web app and activate the function app

In [ ]:
az functionapp Start --name $FunctionName --resource-group $ResourceGroupName

az afd origin update --name functionapp --enabled-state Enabled --host-name dynamichostingfunctionapp.azurewebsites.net --resource-group sustainability-lab-azure --profile-name sustainable-azure-multi-hosting --origin-group-name default
az afd origin update --name webapp --enabled-state Disabled --host-name dynamichostingwebapp.azurewebsites.net --resource-group sustainability-lab-azure --profile-name sustainable-azure-multi-hosting --origin-group-name default
az afd origin list --resource-group sustainability-lab-azure --profile-name sustainable-azure-multi-hosting --origin-group-name default

# Wait

az webapp stop --name $WebAppName --resource-group $ResourceGroupName

## Scale Up

We will deactivate the function app and activate the web app

In [ ]:
az webapp start --name $WebAppName --resource-group $ResourceGroupName

az afd origin update --name webapp --enabled-state Enabled --host-name dynamichostingwebapp.azurewebsites.net --resource-group sustainability-lab-azure --profile-name sustainable-azure-multi-hosting --origin-group-name default
az afd origin update --name functionapp --enabled-state Disabled --host-name dynamichostingfunctionapp.azurewebsites.net --resource-group sustainability-lab-azure --profile-name sustainable-azure-multi-hosting --origin-group-name default
az afd origin list --resource-group sustainability-lab-azure --profile-name sustainable-azure-multi-hosting --origin-group-name default

# Wait

az functionapp Stop --name $FunctionName --resource-group $ResourceGroupName